In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
from firebase_admin import db
import datetime 

# Fetch the service account key JSON file contents
cred = credentials.Certificate("face-rec.json")

# Initialize the app with a service account, granting admin privileges
app = firebase_admin.initialize_app(cred, {
    'databaseURL' : 'https://face-rec-7416b.firebaseio.com/faces',
    'storageBucket': 'face-rec-7416b.appspot.com' 
})

def save_image(image):
    
    dt = datetime.today()
    seconds = str(dt.timestamp())
    date =  seconds.split(".")[0]
    
    bucket=storage.bucket()
    blob = bucket.blob(date + "-" + image.split('/')[-1])
    blob.upload_from_filename(image)

    ref = db.reference('faces')
    ref.set({
        'name': image.split('/')[-1].split('.')[0],
        'path': date + "-" + image.split('/')[-1]
    })


In [2]:
# from my_firebase import save_image

In [3]:
from datetime import datetime


In [4]:
import face_recognition
import cv2
import numpy as np
import os


KNOWN_FACES_DIR =  "Trusted_People"
known_face_encodings = []
known_face_names = []
# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

for name in os.listdir(KNOWN_FACES_DIR):

    # Next we load every file of faces of known person
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):

        
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
        encoding = face_recognition.face_encodings(image)[0]

        # Append encodings and name
        known_face_encodings.append(encoding)
        known_face_names.append(name)

In [5]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
img_counter = 0

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    humans = human_csc.detectMultiScale(gray,1.1,4)
    numOfHumans = len(humans)
    
    if numOfHumans > 0:
        
    #     Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # Or instead, use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

                face_names.append(name)

        process_this_frame = not process_this_frame


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


        if "Hoda" in face_names or "Ibrahim" in face_names or "Elon" in face_names:    
            img_name = "images/{}.png".format(face_names[0])
            cv2.imwrite(img_name, frame)

            save_image(img_name)
            break           

    Display the resulting image
    cv2.imshow('Video', frame)

    Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [6]:
print(face_names)

['Ibrahim']
